<a href="https://colab.research.google.com/github/marcoshsq/Stocks_Market_Data_Analysis/blob/main/Extracting_and_Visualizing_Stock_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Extracting and Visualizing Stock Data**

---

### Description:

It's very important for a Data Scientist to know how to collect, analyze and create visualizations with collected data for decision making. 

This project aims to practice such skills, analyzing the stock data from Tesla and GameStop.

In these first two cells we install and import the relevant libraries for the project.

In [ ]:
!pip install yfinance
#!pip install pandas
#!pip install requests
!pip install bs4
#!pip install plotly

In [2]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Define Graphing Function

In this section, we define the function make_graph. The make_graph function takes a dataframe with stock data (dataframe must contain Date and Close columns), a dataframe with revenue data (dataframe must contain Date and Revenue columns), and the name of the stock.

In [3]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data.Date, infer_datetime_format=True), y=stock_data.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data.Date, infer_datetime_format=True), y=revenue_data.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

# Using yfinance to Extract Stock Data (TSLA)

Using the Ticker function, we enter the ticker symbol of the stock we want to extract data on to create a ticker object. The stock is Tesla and its ticker symbol is TSLA.

In [4]:
tesla = yf.Ticker('TSLA')

Using the ticker object and the function history, we extract stock information and save it in a dataframe named tesla_data. 

Then we set the period parameter to max so we get information for the maximum amount of time.

In [5]:
tesla_data = tesla.history(period="max")

Now we reset the index using the reset_index(inplace=True) function on the tesla_data DataFrame and display the first five rows of the tesla_data dataframe using the head function.

In [6]:
tesla_data.reset_index(inplace=True)
tesla_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29,3.800,5.000,3.508,4.778,93831500,0,0.0
1,2010-06-30,5.158,6.084,4.660,4.766,85935500,0,0.0
2,2010-07-01,5.000,5.184,4.054,4.392,41094000,0,0.0
3,2010-07-02,4.600,4.620,3.742,3.840,25699000,0,0.0
4,2010-07-06,4.000,4.000,3.166,3.222,34334500,0,0.0


# Using Webscraping to Extract Tesla Revenue Data (TSLA)

We'll use the requests library to download the webpage [Tesla Revenue 2010-2021 | TSLA](https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue). Then save the text of the response as a variable named html_data.

In [7]:
url = 'https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue'
html_data = requests.get(url).text

Now we parse the html data using beautiful_soup.

In [8]:
soup = BeautifulSoup(html_data,"html5lib")

Using beautiful soup we extract the table with Tesla Quarterly Revenue and store it into a dataframe named tesla_revenue.

For this project our dataframe will have columns Date and Revenue, and we'll remove the comma and dollar sign from the Revenue column.

In [9]:
tesla_revenue = pd.DataFrame(columns=['Date', 'Revenue'])

for table in soup.find_all('table'):

    if ('Tesla Quarterly Revenue' in table.find('th').text):
        rows = table.find_all('tr')
        
        for row in rows:
            col = row.find_all('td')
            
            if col != []:
                date = col[0].text
                revenue = col[1].text.replace(',','').replace('$','')

                tesla_revenue = tesla_revenue.append({"Date":date, "Revenue":revenue}, ignore_index=True)

We removed the rows in the dataframe that are empty strings or are NaN in the Revenue column. And then printed the entire tesla_revenue DataFrame.

In [10]:
tesla_revenue

,Date,Revenue
0,2021-12-31,17719
1,2021-09-30,13757
2,2021-06-30,11958
3,2021-03-31,10389
4,2020-12-31,10744
5,2020-09-30,8771
6,2020-06-30,6036
7,2020-03-31,5985
8,2019-12-31,7384
9,2019-09-30,6303


In [22]:
tesla_revenue = tesla_revenue[tesla_revenue['Revenue'].astype(bool)]

Here we'll display the last 5 row of the tesla_revenue dataframe using the tail function.

In [23]:
tesla_revenue.tail()

,Date,Revenue
45,2010-09-30,31
46,2010-06-30,28
47,2010-03-31,21
49,2009-09-30,46
50,2009-06-30,27


# Using yfinance to Extract Stock Data (GME)

We'll repeat the process using the Ticker function. But this time we want GameStop stock data, and its ticker symbol is GME.

In [13]:
gme = yf.Ticker('GME')

Using the ticker object and the function history, we extract stock information and save it in a dataframe named gme_data. Once again we set the period parameter to max so we get information for the maximum amount of time.

In [14]:
gme_data = gme.history(period='max')

We reset the index using the reset_index(inplace=True) function on the gme_data DataFrame and display the first five rows of the gme_data dataframe using the head function.

In [15]:
gme_data.reset_index(inplace=True)
gme_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2002-02-13,6.480514,6.773400,6.413184,6.766667,19054000,0.0,0.0
1,2002-02-14,6.850830,6.864296,6.682505,6.733002,2755400,0.0,0.0
2,2002-02-15,6.733001,6.749833,6.632006,6.699336,2097400,0.0,0.0
3,2002-02-19,6.665670,6.665670,6.312187,6.430015,1852600,0.0,0.0
4,2002-02-20,6.463681,6.648839,6.413183,6.648839,1723200,0.0,0.0


# Using Webscraping to Extract GME Revenue Data (GME)

We'll use the requests library to download the webpage [GameStop Revenue 2010-2021 | GME
](https://www.macrotrends.net/stocks/charts/GME/gamestop/revenue?cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork-23455606&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork-23455606&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork-23455606&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork-23455606&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ). And save the text of the response as a variable named html_data.

In [16]:
url = 'https://www.macrotrends.net/stocks/charts/GME/gamestop/revenue'
html_data = requests.get(url).text

Now we parse the html data using beautiful_soup.

In [17]:
soup = BeautifulSoup(html_data,"html5lib")

Using beautiful soup we extract the table with GameStop Quarterly Revenue and store it into a dataframe named gme_revenue.

For this project our dataframe will have columns Date and Revenue, and we'll remove the comma and dollar sign from the Revenue column.

In [18]:
gme_revenue = pd.DataFrame(columns=['Date', 'Revenue'])

for table in soup.find_all('table'):

    if ('GameStop Quarterly Revenue' in table.find('th').text):
        rows = table.find_all('tr')
        
        for row in rows:
            col = row.find_all('td')
            
            if col != []:
                date = col[0].text
                revenue = col[1].text.replace(',','').replace('$','')

                gme_revenue = gme_revenue.append({"Date":date, "Revenue":revenue}, ignore_index=True)

We display the last five rows of the gme_revenue dataframe using the tail function.

In [19]:
gme_revenue.tail()

,Date,Revenue
47,2010-01-31,3524
48,2009-10-31,1835
49,2009-07-31,1739
50,2009-04-30,1981
51,2009-01-31,3492


# Plotting the Tesla Stock Graph

Now we're going to plot a graph with the information we colected, for this we'll use the make_graph function to graph the Tesla Stock Data. The structure to call the make_graph function is make_graph(tesla_data, tesla_revenue, 'Tesla')

In [20]:
make_graph(tesla_data[['Date','Close']], tesla_revenue, 'Tesla')

# Plotting the GameStop Stock Graph

We'll repeat the last step here. The structure to call the make_graph function is make_graph(gme_data, gme_revenue, 'GameStop').

In [21]:
make_graph(gme_data[['Date','Close']], gme_revenue, 'GameStop')

### Project developed by marcoshsq (づ｡◕‿‿◕｡)づ